# Sample Notebook
This is a sample Jupyter Notebook uploaded to GitHub.

In [2]:
# 1. GPU check
!nvidia-smi

# 2. Install dependencies
!pip -q install -U transformers datasets sentencepiece accelerate huggingface_hub python-dotenv feedparser beautifulsoup4 sacrebleu rouge-score gradio



# 3. Clone or pull repo
from getpass import getpass
token = getpass("Enter your GitHub PAT: ")
USERNAME = "sukritichawla"
REPO = "legaldocs_ST"
REPO_URL = f"https://{token}@github.com/{USERNAME}/{REPO}.git"
!git clone $REPO_URL


# 4. Go inside repo
%cd /content/legaldocs_ST

# 5. Hugging Face login
from huggingface_hub import login
login()

Fri Sep 19 13:12:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import pipeline

# ---------------- 1️⃣ Instruction-following model ----------------
simplifier = pipeline("text2text-generation", model="google/flan-t5-base")

# ---------------- 2️⃣ General cleanup dictionary ----------------
general_cleanup_dict = {
    # Rights and obligations
    "reserves the unilateral right to": "may",
    "reserves the right to": "may",
    "shall be entitled to": "can",
    "is entitled to": "can",
    "shall have the right to": "can",
    "may at its sole discretion": "can",
    "is authorized to": "can",
    "has the right to": "can",

    # Actions
    "amend, revise, or update": "change",
    "amend or update": "change",
    "terminated earlier": "ended earlier",
    "shall be terminated": "can be ended",
    "may at any time": "anytime",
    "take any action": "act",
    "perform any obligations": "do its duties",
    "exercise any rights": "use its rights",

    # Agreement/contract terms
    "shall become effective": "starts",
    "shall remain in full force and effect": "will continue",
    "hereinafter referred to as": "called",
    "notwithstanding the foregoing": "despite this",
    "pursuant to": "under",
    "subject to": "under",
    "prior written notice": "written notice",
    "herein": "in this document",
    "hereof": "of this document",
    "hereunder": "under this",
    "for the term of": "for",
    "during the term": "while active",
    "from time to time": "occasionally",

    # Acceptance/consent
    "shall constitute acceptance": "means you accept",
    "constitutes acceptance": "means you accept",
    "continued use of the service": "using the service",
    "by continuing to use": "by using",
    "shall be deemed to": "means",
    "is deemed to": "means",
    "consent to": "agree to",

    # Misc legalese
    "breach of terms": "breaking the rules",
    "explicitly terminated by either party": "ended by either party",
    "unless otherwise agreed": "unless agreed otherwise",
    "at any other reasonable time": "anytime",
    "in accordance with": "under",
    "as soon as practicable": "quickly",
    "without prejudice to": "without affecting",
    "to the extent permitted by law": "if allowed by law",
    "for the avoidance of doubt": "to be clear",
    "including but not limited to": "including",
    "any and all": "all",
    "each and every": "every",
    "without limitation": "including",
    "in the event of": "if",
    "as applicable": "if relevant",
    "as required by law": "if law requires",
    "without notice": "without informing",
    "at its discretion": "as it decides",
    "in perpetuity": "forever",
    "for all purposes": "for all uses",
    "to the fullest extent permitted by law": "as much as allowed by law",
    "not limited to": "including",
}

# ---------------- 3️⃣ Simplification function ----------------
def simplify_legal_text(text):
    """
    Simplify any legal text into short, plain English.
    Keeps key details like dates, obligations, notice periods.
    """
    # Step 1: Instruction prompt
    prompt = (
        "Rewrite this legal text in short, plain English, "
        "keeping all important details such as dates, obligations, and notice periods. "
        "Avoid legal jargon and make it concise:\n\n" + text
    )

    # Step 2: Model generates simplified text
    result = simplifier(
        prompt,
        max_new_tokens=80,
        do_sample=False
    )[0]["generated_text"]

    # Step 3: Apply general cleanup dictionary
    for k, v in general_cleanup_dict.items():
        result = result.replace(k, v)

    # Step 4: Clean extra spaces
    result = " ".join(result.split())
    return result.strip()

# ---------------- 4️⃣ Example usage ----------------
legal_texts = [
    """Users are solely responsible for ensuring that their use of the service, including all uploaded, shared, or transmitted content, complies with applicable local, state, and federal laws, and the company reserves the right to cooperate with legal authorities in any investigation.""",

    """All intellectual property rights, including but not limited to trademarks, logos, copyrights, patents, and proprietary software, shall remain the exclusive property of the company or its licensors, and users shall not claim any ownership or usage rights beyond what is expressly granted.""",

    """No waiver of any provision of this agreement shall constitute a continuing waiver, and any failure by the company to enforce a provision shall not prevent future enforcement of the same or other provisions.""",

    """In accordance with the applicable law, all parties shall comply with their obligations and any breach of these terms may result in remedies including but not limited to damages or termination of this agreement.""",

    """The service provider reserves the unilateral right to amend, revise, or update these terms and policies annually on 01/01 or at any other reasonable time with thirty (30) days’ notice, and continued use of the service shall constitute acceptance of such amendments.""",

    """This agreement shall become effective on 01/09/2025 and shall remain in full force and effect until explicitly terminated by either party with sixty (60) days’ prior written notice, unless terminated earlier pursuant to breach of terms.""",

    """Users must immediately notify the service provider of any unauthorized access or security breach relating to their account, and failure to do so within forty-eight (48) hours may limit the provider’s liability.""",

    """Invoices issued by the service provider shall be payable within fifteen (15) business days from the date of issuance; any failure to remit payment within this period shall accrue interest at a rate of 1.5% per month until paid in full.""",

    """The company shall not be liable for any indirect, incidental, special, or consequential damages arising from use or inability to use the service, even if the company has been advised of the possibility of such damages.""",

    """All confidential information shared between the parties must be safeguarded for a minimum period of seven (7) years from the date of disclosure, and may only be used for purposes explicitly authorized under this agreement, unless otherwise required by law."""
]


for t in legal_texts:
    simplified = simplify_legal_text(t)
    print("\n--- Simplified Text ---\n")
    print(simplified)


Device set to use cuda:0



--- Simplified Text ---

Users are solely responsible for ensuring that their use of the Service, including all uploaded, shared, or transmitted content, complies with applicable local, state, and federal laws, and the company may cooperate with legal authorities in any investigation.

--- Simplified Text ---

All intellectual property rights, including trademarks, logos, copyrights, patents, and proprietary software, shall remain the exclusive property of the company or its licensors, and users shall not claim any ownership or usage rights beyond what is expressly granted.

--- Simplified Text ---

No waiver of any provision of this Agreement shall constitute a continuing waiver, and any failure by the Company to enforce a provision shall not prevent future enforcement of the same or other provisions.

--- Simplified Text ---

In accordance with the applicable law, all parties shall comply with their obligations and any breach of these terms may result in remedies including damages o

In [53]:
!pip install transformers datasets evaluate accelerate sentencepiece sacremoses torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.9 MB/s eta 0:00:00


In [54]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 37.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [26]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00


In [61]:
import re
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import evaluate

# Max lengths
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

# Keywords to preserve meaning
KEYWORDS_TO_PRESERVE = [
    "shall", "liable", "jurisdiction", "indemnify", "arbitration",
    "agreement", "contract", "party", "lease", "tenant", "landlord"
]

# Output directory
OUTPUT_DIR = "./legal_simplifier_model"


In [56]:
# Replace with your full 100 pairs
EXAMPLE_PAIRS = [
    {"complex": "Every consumer shall have the right to be protected against hazardous products.",
     "simple": "Consumers have the right to be safe from harmful products."},
    {
        "complex": "The party of the first part hereby covenants and agrees to indemnify and hold harmless the party of the second part from any and all claims, liabilities, damages, or expenses arising from the use of the premises.",
        "simple": "The first party agrees to protect the second party from any claims, damages, or costs caused by using the property."
    },
    {
        "complex": "In the event of a dispute, the matter shall be submitted to arbitration in accordance with the rules of the American Arbitration Association, and judgment upon the award rendered by the arbitrator(s) may be entered in any court having jurisdiction thereof.",
        "simple": "If there is a dispute, it will be resolved by arbitration under AAA rules. The arbitrator's decision can be enforced by a court."
    },
    {
        "complex": "The lessee shall not, without prior written consent of the lessor, assign, sublet, or otherwise transfer any interest in this lease or the premises.",
        "simple": "The tenant must get the landlord's written permission before subletting or transferring the lease."
    },
   {
        "complex": "This Agreement shall be governed by and construed in accordance with the laws of the State of New York, without regard to its conflict of law provisions.",
        "simple": "This agreement is governed by New York law, ignoring its conflict-of-law rules."
    },
    {
        "complex": "Failure of either party to enforce any provision of this Agreement shall not constitute a waiver of such provision or of any other provision.",
        "simple": "If either party does not enforce a part of this agreement, it does not mean they give up the right to enforce it later."
    },
    {
        "complex": "No amendment, modification, or addition to this Agreement shall be binding unless set forth in writing and signed by both parties.",
        "simple": "Changes to this agreement must be written and signed by both parties to be valid."
    },
    {
        "complex": "The contractor shall, at its own expense, obtain and maintain all permits, licenses, and approvals required for the performance of its obligations under this Agreement.",
        "simple": "The contractor must get and keep all permits and licenses needed to fulfill this agreement, at its own cost."
    },
    {
        "complex": "Each party shall maintain the confidentiality of all proprietary information disclosed by the other party and shall not disclose such information to any third party without prior written consent.",
        "simple": "Both parties must keep shared confidential information secret and cannot share it with others without written permission."
    },
    {
        "complex": "This Agreement constitutes the entire agreement between the parties with respect to the subject matter hereof and supersedes all prior agreements and understandings, whether written or oral.",
        "simple": "This document is the full agreement between the parties and replaces any earlier written or spoken agreements."
    },
    {
        "complex": "The employee shall be entitled to a severance package equal to three months’ salary in the event of termination without cause.",
        "simple": "If the employee is fired without cause, they will receive three months' salary as severance pay."
    },
    {
        "complex": "The borrower shall repay the principal amount together with accrued interest on or before the maturity date specified herein.",
        "simple": "The borrower must repay the loan and interest by the stated maturity date."
    },
    {
        "complex": "Time is of the essence with respect to all obligations of the parties under this Agreement.",
        "simple": "Deadlines in this agreement are very important and must be followed."
    },
    {
        "complex": "The vendor shall promptly notify the purchaser in writing of any defects discovered in the goods and shall cooperate with the purchaser in remedying such defects.",
        "simple": "The vendor must quickly inform the buyer in writing of any defects and work with them to fix the problem."
    },
    {
        "complex": "The guarantor hereby unconditionally guarantees the full and punctual payment of all obligations of the debtor under this Agreement.",
        "simple": "The guarantor promises to pay all the debtor's obligations on time if the debtor does not."
    },
    {
        "complex": "Neither party shall be liable for any failure or delay in performance due to causes beyond its reasonable control, including acts of God, war, or natural disasters.",
        "simple": "Neither party is responsible for delays or failures caused by events outside their control, like natural disasters or war."
    },
    {
        "complex": "The obligations set forth in this Section shall survive the termination or expiration of this Agreement.",
        "simple": "These obligations continue even after this agreement ends or expires."
    },
    {
        "complex": "The supplier shall deliver the goods FOB shipping point, and title and risk of loss shall pass to the buyer upon delivery to the carrier.",
        "simple": "The supplier delivers the goods at the shipping point. Ownership and risk pass to the buyer once the goods are given to the carrier."
    },
    {
        "complex": "Any person who defaults in the payment of income tax within the time specified under the Income Tax Act, 1961, shall be liable to pay simple interest at the rate of 1% per month or part of a month on the amount of such tax from the date it was due until the date of actual payment.",
        "simple": "If you miss paying your income tax on time, you will have to pay 1% extra per month on the pending amount until you pay it."
    },
    {
        "complex": "The tenant shall not, without the previous consent in writing of the landlord, sub-let, assign, or part with the possession of the premises or any part thereof.",
        "simple": "A tenant cannot rent out or give the house to someone else unless the landlord agrees in writing."
    },
    {
        "complex": "A marriage may be solemnized between any two Hindus if neither party has a spouse living at the time of the marriage and both parties are capable of giving valid consent.",
        "simple": "A Hindu marriage is valid if both people are unmarried and agree to the marriage."
    },
    {
        "complex": "Every consumer shall have the right to be protected against the marketing of goods, products or services which are hazardous to life and property.",
        "simple": "Consumers have the right to be safe from harmful products and services."
    },
    {
        "complex": "Any dispute between landlord and tenant regarding rent, maintenance, or eviction shall be resolved by the Rent Control Court having jurisdiction in the area.",
        "simple": "If a landlord and tenant have a problem about rent, repairs, or eviction, the case will go to the Rent Control Court."
    },
    {
        "complex": "If a borrower defaults on repayment of a secured loan, the creditor has the right to take possession of the secured asset and sell it to recover the outstanding amount.",
        "simple": "If you don't repay a secured loan, the lender can take and sell the property kept as security to recover the money."
    },
    {
        "complex": "No person shall engage in insider trading of securities, and any violation shall attract penalties including imprisonment which may extend to five years or with fine which shall not be less than five lakh rupees.",
        "simple": "Insider trading (using secret company info to trade shares) is illegal and can lead to jail time or a fine of at least five lakh rupees."
    },
    {
        "complex": "The landlord shall be bound to keep the premises in a good and tenantable repair and make such repairs as may be necessary to keep the premises in a proper state for use by the tenant.",
        "simple": "The landlord must keep the house in good condition and do necessary repairs for the tenant to live properly."
    },
    {
        "complex": "Where a person dies intestate, the property of the deceased shall devolve according to the personal law applicable to the individual at the time of death.",
        "simple": "If a person dies without a will, their property will be distributed according to their religion’s personal law."
    },
    {
        "complex": "Every banking company shall maintain at all times a cash reserve ratio with the Reserve Bank of India as prescribed, failing which it shall be liable to pay penal interest.",
        "simple": "Banks must keep a fixed amount of money with the RBI. If they fail, they have to pay a penalty."
    },
    {
        "complex": "Any person who defaults in the payment of income tax within the time specified under the Income Tax Act, 1961, shall be liable to pay simple interest at the rate of 1% per month or part of a month on the amount of such tax from the date it was due until the date of actual payment.",
        "simple": "If you miss paying your income tax on time, you will have to pay 1% extra per month on the pending amount until you pay it."
    },
    {
        "complex": "The tenant shall not, without the previous consent in writing of the landlord, sub-let, assign, or part with the possession of the premises or any part thereof.",
        "simple": "A tenant cannot rent out or give the house to someone else unless the landlord agrees in writing."
    },
    {
        "complex": "A marriage may be solemnized between any two Hindus if neither party has a spouse living at the time of the marriage and both parties are capable of giving valid consent.",
        "simple": "A Hindu marriage is valid if both people are unmarried and agree to the marriage."
    },
    {
        "complex": "Every consumer shall have the right to be protected against the marketing of goods, products or services which are hazardous to life and property.",
        "simple": "Consumers have the right to be safe from harmful products and services."
    },
    {
        "complex": "Any dispute between landlord and tenant regarding rent, maintenance, or eviction shall be resolved by the Rent Control Court having jurisdiction in the area.",
        "simple": "If a landlord and tenant have a problem about rent, repairs, or eviction, the case will go to the Rent Control Court."
    },
    {
        "complex": "If a borrower defaults on repayment of a secured loan, the creditor has the right to take possession of the secured asset and sell it to recover the outstanding amount.",
        "simple": "If you don't repay a secured loan, the lender can take and sell the property kept as security to recover the money."
    },
    {
        "complex": "No person shall engage in insider trading of securities, and any violation shall attract penalties including imprisonment which may extend to five years or with fine which shall not be less than five lakh rupees.",
        "simple": "Insider trading (using secret company info to trade shares) is illegal and can lead to jail time or a fine of at least five lakh rupees."
    },
    {
        "complex": "The landlord shall be bound to keep the premises in a good and tenantable repair and make such repairs as may be necessary to keep the premises in a proper state for use by the tenant.",
        "simple": "The landlord must keep the house in good condition and do necessary repairs for the tenant to live properly."
    },
    {
        "complex": "Where a person dies intestate, the property of the deceased shall devolve according to the personal law applicable to the individual at the time of death.",
        "simple": "If a person dies without a will, their property will be distributed according to their religion’s personal law."
    },
    {
        "complex": "Every banking company shall maintain at all times a cash reserve ratio with the Reserve Bank of India as prescribed, failing which it shall be liable to pay penal interest.",
        "simple": "Banks must keep a fixed amount of money with the RBI. If they fail, they have to pay a penalty."
    },
    {
        "complex": "No employee shall be required to work for more than forty-eight hours in any week except in cases of overtime for which extra wages shall be paid as per the Factories Act, 1948.",
        "simple": "Workers cannot be made to work more than 48 hours a week. If they do extra work, they must be paid extra."
    },
    {
        "complex": "If the tenant fails to pay rent for a continuous period of two months, the landlord shall have the right to file for eviction under the applicable tenancy laws.",
        "simple": "If a tenant does not pay rent for two months in a row, the landlord can legally ask them to leave."
    },
    {
        "complex": "Any agreement entered into by a minor is void ab initio under the Indian Contract Act, 1872.",
        "simple": "Any contract made by a person under 18 is not valid."
    },
    {
        "complex": "Every employer shall provide for the payment of maternity benefit to women employees at the rate of the average daily wage for the period of her actual absence.",
        "simple": "Employers must pay women their usual daily wage during maternity leave."
    },
    {
        "complex": "Where an arbitration agreement exists, the parties shall submit their dispute to arbitration and not to a civil court, as per the Arbitration and Conciliation Act, 1996.",
        "simple": "If there is an arbitration agreement, disputes must be settled through arbitration, not in court."
    },
    {
        "complex": "Every person shall have the right to information accessible under the control of public authorities consistent with the provisions of the Right to Information Act, 2005.",
        "simple": "Everyone has the right to ask for information from government offices under the RTI Act."
    },
    {
        "complex": "The employer shall be liable to pay compensation if an employee is injured by accident arising out of and in the course of his employment as per the Employees’ Compensation Act, 1923.",
        "simple": "If an employee is injured while working, the employer must pay compensation."
    },
    {
        "complex": "If a cheque is dishonoured due to insufficient funds, the drawer shall be deemed to have committed an offence and shall be punishable with imprisonment up to two years or fine up to twice the amount of the cheque.",
        "simple": "If your cheque bounces because of not enough money, you can face jail time or pay a fine up to double the cheque amount."
    },
    {
        "complex": "No employer shall terminate the services of a workman who has been in continuous service for not less than one year, without giving one month’s notice or wages in lieu of notice.",
        "simple": "An employee who worked for at least one year cannot be fired without a one-month notice or one-month salary."
    },
    {
        "complex": "Where a person makes a false statement in a declaration under oath, such person shall be punishable for perjury under the Indian Penal Code.",
        "simple": "If someone lies in a sworn statement, they can be punished for perjury under law."
    },
    {
        "complex": "Any person who defaults in the payment of income tax within the time specified under the Income Tax Act, 1961, shall be liable to pay simple interest at the rate of 1% per month or part of a month on the amount of such tax from the date it was due until the date of actual payment.",
        "simple": "If you miss paying your income tax on time, you will have to pay 1% extra per month on the pending amount until you pay it."
    },
    {
        "complex": "The tenant shall not, without the previous consent in writing of the landlord, sub-let, assign, or part with the possession of the premises or any part thereof.",
        "simple": "A tenant cannot rent out or give the house to someone else unless the landlord agrees in writing."
    },
    {
        "complex": "A marriage may be solemnized between any two Hindus if neither party has a spouse living at the time of the marriage and both parties are capable of giving valid consent.",
        "simple": "A Hindu marriage is valid if both people are unmarried and agree to the marriage."
    },
    {
        "complex": "Every consumer shall have the right to be protected against the marketing of goods, products or services which are hazardous to life and property.",
        "simple": "Consumers have the right to be safe from harmful products and services."
    },
    {
        "complex": "Any dispute between landlord and tenant regarding rent, maintenance, or eviction shall be resolved by the Rent Control Court having jurisdiction in the area.",
        "simple": "If a landlord and tenant have a problem about rent, repairs, or eviction, the case will go to the Rent Control Court."
    },
    {
        "complex": "If a borrower defaults on repayment of a secured loan, the creditor has the right to take possession of the secured asset and sell it to recover the outstanding amount.",
        "simple": "If you don't repay a secured loan, the lender can take and sell the property kept as security to recover the money."
    },
    {
        "complex": "No person shall engage in insider trading of securities, and any violation shall attract penalties including imprisonment which may extend to five years or with fine which shall not be less than five lakh rupees.",
        "simple": "Insider trading (using secret company info to trade shares) is illegal and can lead to jail time or a fine of at least five lakh rupees."
    },
    {
        "complex": "The landlord shall be bound to keep the premises in a good and tenantable repair and make such repairs as may be necessary to keep the premises in a proper state for use by the tenant.",
        "simple": "The landlord must keep the house in good condition and do necessary repairs for the tenant to live properly."
    },
    {
        "complex": "Where a person dies intestate, the property of the deceased shall devolve according to the personal law applicable to the individual at the time of death.",
        "simple": "If a person dies without a will, their property will be distributed according to their religion’s personal law."
    },
    {
        "complex": "Every banking company shall maintain at all times a cash reserve ratio with the Reserve Bank of India as prescribed, failing which it shall be liable to pay penal interest.",
        "simple": "Banks must keep a fixed amount of money with the RBI. If they fail, they have to pay a penalty."
    },
    {
        "complex": "No employee shall be required to work for more than forty-eight hours in any week except in cases of overtime for which extra wages shall be paid as per the Factories Act, 1948.",
        "simple": "Workers cannot be made to work more than 48 hours a week. If they do extra work, they must be paid extra."
    },
    {
        "complex": "If the tenant fails to pay rent for a continuous period of two months, the landlord shall have the right to file for eviction under the applicable tenancy laws.",
        "simple": "If a tenant does not pay rent for two months in a row, the landlord can legally ask them to leave."
    },
    {
        "complex": "Any agreement entered into by a minor is void ab initio under the Indian Contract Act, 1872.",
        "simple": "Any contract made by a person under 18 is not valid."
    },
    {
        "complex": "Every employer shall provide for the payment of maternity benefit to women employees at the rate of the average daily wage for the period of her actual absence.",
        "simple": "Employers must pay women their usual daily wage during maternity leave."
    },
    {
        "complex": "Where an arbitration agreement exists, the parties shall submit their dispute to arbitration and not to a civil court, as per the Arbitration and Conciliation Act, 1996.",
        "simple": "If there is an arbitration agreement, disputes must be settled through arbitration, not in court."
    },
    {
        "complex": "Every person shall have the right to information accessible under the control of public authorities consistent with the provisions of the Right to Information Act, 2005.",
        "simple": "Everyone has the right to ask for information from government offices under the RTI Act."
    },
    {
        "complex": "The employer shall be liable to pay compensation if an employee is injured by accident arising out of and in the course of his employment as per the Employees’ Compensation Act, 1923.",
        "simple": "If an employee is injured while working, the employer must pay compensation."
    },
    {
        "complex": "If a cheque is dishonoured due to insufficient funds, the drawer shall be deemed to have committed an offence and shall be punishable with imprisonment up to two years or fine up to twice the amount of the cheque.",
        "simple": "If your cheque bounces because of not enough money, you can face jail time or pay a fine up to double the cheque amount."
    },
    {
        "complex": "No employer shall terminate the services of a workman who has been in continuous service for not less than one year, without giving one month’s notice or wages in lieu of notice.",
        "simple": "An employee who worked for at least one year cannot be fired without a one-month notice or one-month salary."
    },
    {
        "complex": "Where a person makes a false statement in a declaration under oath, such person shall be punishable for perjury under the Indian Penal Code.",
        "simple": "If someone lies in a sworn statement, they can be punished for perjury under law."
    },
    {
        "complex": "Section 100 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 100, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 101 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 101, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 102 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 102, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 103 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 103, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 104 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 104, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 105 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 105, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 106 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 106, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 107 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 107, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 108 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 108, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 109 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 109, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 110 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 110, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 111 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 111, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 112 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 112, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 113 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 113, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 114 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 114, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 115 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 115, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 116 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 116, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 117 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 117, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 118 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 118, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 119 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 119, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 120 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 120, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 121 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 121, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 122 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 122, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 123 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 123, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 124 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 124, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 125 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 125, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 126 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 126, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 127 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 127, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 128 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 128, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 129 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 129, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 130 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 130, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 131 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 131, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 132 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 132, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 133 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 133, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 134 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 134, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 135 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 135, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 136 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 136, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 137 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 137, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 138 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 138, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 139 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 139, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 140 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 140, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 141 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 141, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 142 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 142, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 143 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 143, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 144 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 144, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 145 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 145, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 146 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 146, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 147 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 147, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 148 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 148, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 149 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 149, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 150 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 150, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 151 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 151, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 152 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 152, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 153 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 153, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 154 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 154, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 155 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 155, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 156 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 156, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 157 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 157, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 158 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 158, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 159 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 159, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 160 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 160, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 161 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 161, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 162 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 162, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 163 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 163, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 164 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 164, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 165 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 165, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 166 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 166, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 167 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 167, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 168 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 168, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 169 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 169, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 170 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 170, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 171 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 171, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 172 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 172, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 173 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 173, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 174 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 174, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 175 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 175, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 176 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 176, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 177 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 177, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 178 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 178, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Section 179 of the Sample Act states that any person found guilty of violating this provision shall be punishable with fine which may extend to fifty thousand rupees or imprisonment up to six months.",
        "simple": "If you break the rule in section 179, you may have to pay up to 50,000 rupees or go to jail for six months."
    },
    {
        "complex": "Any person who wilfully conceals income or furnishes inaccurate particulars under the Income Tax Act shall be liable for penalty in addition to tax payable.",
        "simple": "If you hide income or give wrong info for taxes, you must pay a penalty on top of the taxes owed."
    },
    {
        "complex": "The landlord may enter the rented premises only after giving reasonable notice to the tenant and during normal hours unless in case of emergency.",
        "simple": "A landlord can visit the rented house only after giving notice and during normal hours, except for emergencies."
    },
    {
        "complex": "A marriage between parties within the prohibited degrees of relationship as specified in the Hindu Marriage Act is voidable at the option of either party.",
        "simple": "Marriages between close relatives are invalid if one person chooses to cancel it."
    },
    {
        "complex": "No product shall be sold to consumers with misleading or false representations, and any violation shall attract penalties under the Consumer Protection Act, 2019.",
        "simple": "Selling products with false or misleading info is illegal and punishable under consumer law."
    },
    {
        "complex": "If a lessee fails to maintain the leased property in good condition, the lessor may terminate the lease and claim damages for any loss caused.",
        "simple": "If a tenant damages the property, the landlord can end the lease and ask for compensation."
    },
    {
        "complex": "Any company which fails to maintain the minimum net worth as prescribed under the Companies Act, 2013, shall be liable to penalties and may face restrictions on business operations.",
        "simple": "If a company doesn’t keep the required minimum capital, it can be fined and restricted from doing business."
    },
    {
        "complex": "No employer shall discriminate against any employee on the basis of religion, caste, sex, or disability, and any such discrimination shall be punishable under law.",
        "simple": "Employers cannot treat workers unfairly because of religion, caste, gender, or disability."
    },
    {
        "complex": "The Information Technology Act, 2000 penalizes unauthorized access, data theft, and hacking, with imprisonment up to three years or fine up to five lakh rupees.",
        "simple": "Hacking, stealing data, or accessing systems without permission can lead to jail up to 3 years or a fine up to 5 lakh rupees."
    },
    {
        "complex": "Every citizen has the right to approach the court for enforcement of Fundamental Rights guaranteed under the Constitution of India.",
        "simple": "Anyone can go to court to protect their basic rights under the Indian Constitution."
    },
    {
        "complex": "Any person discharging pollutants into a water body without prior consent from the Pollution Control Board shall be liable to penalties and remedial action.",
        "simple": "If someone pollutes water without permission, they can be fined and forced to clean it up."
    },
    {
        "complex": "No individual shall be held liable for defamation unless it is proven that the statement was made with malicious intent or reckless disregard for truth.",
        "simple": "A person is guilty of defamation only if they spread lies with harmful intent."
    },
    {
        "complex": "Where an employee suffers occupational disease due to work conditions, the employer is required to pay medical expenses and compensation under the Employee Compensation Act.",
        "simple": "If a worker gets sick from work, the employer must pay for treatment and compensation."
    },
    {
        "complex": "Any person who knowingly sells adulterated food shall be punishable with imprisonment and fine as per the Food Safety and Standards Act, 2006.",
        "simple": "Selling contaminated or fake food knowingly can lead to jail and fines."
    },
    {
        "complex": "A contract for sale of goods is void if it is entered into by coercion, undue influence, fraud, misrepresentation, or mistake as per the Indian Contract Act, 1872.",
        "simple": "A sales contract is invalid if it was made using force, fraud, lies, or a mistake."
    },
    {
        "complex": "Any person obstructing the rights of persons with disabilities under the Rights of Persons with Disabilities Act, 2016, shall be liable to penalties and imprisonment.",
        "simple": "Blocking or mistreating people with disabilities is illegal and can lead to fines or jail."
    },
    {
        "complex": "The Prevention of Money Laundering Act, 2002 provides that any person involved in converting, transferring, or acquiring proceeds of crime shall be punished with imprisonment and fine.",
        "simple": "If someone handles money from crimes, they can go to jail and pay fines."
    },
    {
        "complex": "Any employer failing to provide a safe working environment and protective equipment shall be liable under the Factories Act, 1948.",
        "simple": "Employers must give safe workplaces and protective gear, or they can be punished."
    },
    {
        "complex": "Where an intellectual property right is infringed, the owner may approach the civil or criminal court for injunctions, damages, or other remedies.",
        "simple": "If someone steals your intellectual property, you can go to court to stop it and get compensation."
    },
    {
        "complex": "Any person obstructing the free exercise of religion or religious processions shall be liable under relevant provisions of the Indian Penal Code.",
        "simple": "Stopping someone from practicing their religion or holding a religious event is illegal."
    },
    {
        "complex": "Any financial institution which knowingly provides false statements in annual reports or misrepresents its accounts shall be liable under the Companies Act and SEBI regulations.",
        "simple": "Banks or companies giving false info in reports can be punished under company and SEBI laws."
    },


]

# Load dataset
def load_dataset():
    return Dataset.from_list([{"complex": p["complex"], "simple": p["simple"]} for p in EXAMPLE_PAIRS])

dataset = load_dataset()
dataset = dataset.train_test_split(test_size=0.2, seed=42)


In [47]:
import re

def preprocess(tokenizer, examples):
    # Highlight keywords
    def highlight_keywords(text):
        for kw in KEYWORDS_TO_PRESERVE:
            text = re.sub(rf"\b({kw})\b", r"<keyword>\1</keyword>", text, flags=re.IGNORECASE)
        return text

    # Prepare inputs with improved prompt wording
    inputs = [
        "Simplify this legal clause in plain English but preserve the meaning of keywords: "
        + highlight_keywords(c)
        + " Important keywords: " + ", ".join(KEYWORDS_TO_PRESERVE)
        for c in examples["complex"]
    ]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length"  # keeps batch shapes consistent
    )

    labels = tokenizer(
        examples["simple"],
        max_length=MAX_TARGET_LENGTH,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [48]:
MODEL_NAME = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenized_datasets = dataset.map(
    lambda x: preprocess(tokenizer=tokenizer, examples=x),
    batched=True
)


Map:   0%|          | 0/134 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

In [73]:
import re

def preprocess(tokenizer, examples):
    inputs = [
        "Simplify this legal clause in plain English: " + c
        for c in examples["complex"]
    ]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        examples["simple"],
        max_length=MAX_TARGET_LENGTH,
        truncation=True,
        padding="max_length"
    )

    # Replace padding token IDs with -100 for correct loss computation
    labels["input_ids"] = [
        [(lid if lid != tokenizer.pad_token_id else -100) for lid in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [74]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [77]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # correct argument name
    save_strategy="epoch",        # optional: save checkpoints every epoch
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    predict_with_generate=True,
    report_to="none"
)



In [62]:
pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.2 MB/s eta 0:00:00


In [78]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer)
)

trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


/tmp/ipython-input-134597894.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Bleu,Keyword Preservation,Pred Len,Label Len
1,No log,4.215798,0.107641,0.985294,6.794118,19.705882
2,No log,2.585231,0.000000,0.823529,0.000000,19.705882
3,No log,1.172735,0.000000,0.823529,0.000000,19.705882
4,No log,0.787397,0.000000,0.823529,0.000000,19.705882
5,No log,0.719140,0.000000,0.823529,0.000000,19.705882


('./legal_simplifier_model/tokenizer_config.json',
 './legal_simplifier_model/special_tokens_map.json',
 './legal_simplifier_model/spiece.model',
 './legal_simplifier_model/added_tokens.json',
 './legal_simplifier_model/tokenizer.json')

In [71]:
def simplify(text):
    inputs = tokenizer(
        "Simplify this legal clause in plain English but preserve keywords: " + text,
        return_tensors="pt", truncation=True, max_length=MAX_INPUT_LENGTH
    )
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=MAX_TARGET_LENGTH)

    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Optional post-process to restore missing keywords
    for kw in KEYWORDS_TO_PRESERVE:
        if kw.lower() in text.lower() and kw.lower() not in pred.lower():
            pred = kw + " " + pred
    return pred


In [72]:
test_texts = [
    "Every consumer shall have the right to be protected against the marketing of goods, products or services which are hazardous to life and property.",
    "Every banking company shall maintain at all times a cash reserve ratio with the Reserve Bank of India as prescribed, failing which it shall be liable to pay penal interest.",
    "This Agreement shall be governed by and construed in accordance with the laws of the State of New York, without regard to its conflict of law provisions."
]

for t in test_texts:
    print("ORIGINAL:", t)
    print("PREDICT :", simplify(t))
    print("-"*80)


ORIGINAL: Every consumer shall have the right to be protected against the marketing of goods, products or services which are hazardous to life and property.
PREDICT : shall Consumers have the right to be safe from harmful products and services.
--------------------------------------------------------------------------------
ORIGINAL: Every banking company shall maintain at all times a cash reserve ratio with the Reserve Bank of India as prescribed, failing which it shall be liable to pay penal interest.
PREDICT : liable shall Banks must keep a fixed amount of money with the RBI. If they fail, they will have to pay a penalty.
--------------------------------------------------------------------------------
ORIGINAL: This Agreement shall be governed by and construed in accordance with the laws of the State of New York, without regard to its conflict of law provisions.
PREDICT : This Agreement shall be governed by and construed in accordance with the laws of New York, without regard to its